# 공공데이터 포털 관광지 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from time import sleep

pages = 6
API_key = ''

name_list = []
gugun_list = []
lat_list = []
lng_list = []
place_list = []
add_list = []
tel_list = []
home_list = []
holiday_list = []
image_list = []
sub_list = []
search_list = []

# for page in range(1, 2):
for page in range(1, pages+1):
    url = f'http://apis.data.go.kr/6260000/AttractionService/getAttractionKr?serviceKey={API_key}&pageNo={page}&numOfRows=20&resultType=json'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    n_dict = json.loads(response.text)

#     print(n_dict['getAttractionKr'])
    for area_name in n_dict['getAttractionKr']['item']:
        name_list.append(area_name['MAIN_TITLE'])
        gugun_list.append(area_name['GUGUN_NM'])
        lat_list.append(area_name['LAT'])
        lng_list.append(area_name['LNG'])
        if area_name['PLACE'].count('(') > 0:
            x = area_name['PLACE'].split('(')[0]
            place_list.append(x)
        else:
            place_list.append(area_name['PLACE'])
        add_list.append(area_name['ADDR1'])
        tel_list.append(area_name['CNTCT_TEL'])
        home_list.append(area_name['HOMEPAGE_URL'])
        holiday_list.append(area_name['HLDY_INFO'])
        image_list.append(area_name['MAIN_IMG_NORMAL'])
        sub_list.append(area_name['ITEMCNTNTS'])
        if area_name['PLACE'].count('(') > 0:
            x = "부산 " + area_name['PLACE'].split('(')[0]
            search_list.append(x)
        else:
            search_list.append("부산 " + area_name['PLACE'])
        print(area_name['MAIN_TITLE'], "완료")
        
    sleep(0.33)

In [ ]:
import pandas as pd

columns = ['name','GUN', 'address', 'lat', 'lng', 'tel', 'hompage', 'holiday', 'image', 'explain' ]
df = pd.DataFrame(columns=columns)
for place, gun, address, lat, lng, tel, home, holiday, image, explain in zip(place_list, gugun_list, add_list, lat_list, lng_list, tel_list, home_list, holiday_list, image_list, sub_list):
    row = [place, gun, address, lat, lng, tel, home, holiday, image, explain]
    series = pd.Series(row, index=df.columns)
    df = df.append(series, ignore_index=True)

print(df)

# 카카오맵 리뷰 크롤링

### 라이브러리 호출

In [ ]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
from selenium import webdriver
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import warnings
import math

# 경고 메시지를 무시하고 숨기거나
warnings.filterwarnings(action='ignore')
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')

In [ ]:
def search(place):
    global driver
    
    check = False
    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')
    search_area.send_keys(place)
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)
    
    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.list"]/li[1]')
    except:
        return
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    review = (soup.select('#info\.search\.place\.list > li.PlaceItem.clickArea > div.rating.clickArea > span.score > a'))
#     print(x[0]['href'])
#     review = soup.select('#info\.search\.place\.list > li.PlaceItem.clickArea.PlaceItem-ACTIVE > div.rating.clickArea > span.score > a')
    review_link = review[0]['href']
    
    # 새탭 열기
    driver.execute_script('window.open("about:blank", "_blank");')
    tabs = driver.window_handles
    driver.switch_to.window(tabs[1]) # 탭 이동
    driver.get(review_link)
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 스크롤 끝까지 내리기
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        sleep(2)
        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
        tmp = driver.page_source
        tmp2 = BeautifulSoup(tmp, 'html.parser')

        try:
            review_find = tmp2.select('#mArticle > div.cont_evaluation > strong > span')
            total_page = math.ceil(int(review_find[0].text) / 5)
        except:
            check = True
            break
#         print(total_review)
    
    if check == True:
        pass
    else:
        for i in range(1, total_page+1):
            t1 = driver.page_source
            t2 = BeautifulSoup(t1, "html.parser")
            t3 = t2.find(name="div", attrs={"class":"evaluation_review"})
    #         print(t3)

            star = t3.find_all('em',{'class':'num_rate'})
            review = t3.find_all('p',{'class':'txt_comment'})
            name = t3.find_all('a', {'class': 'link_user'})

    #         names.append(place)
            stars.extend(star)
            reviews.extend(review)
            nicknames.extend(name)

            i += 1

            if i > total_page:
                break

            next_page = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
            next_page.send_keys(Keys.ENTER)
            sleep(1)
    
    
stars = []
reviews = []
names = []
nicknames = []
    
# driver.implicitly_wait(4) 
# driver.get('https://map.kakao.com/')

# search('스포원파크')
for tour_name in search_list:
# for tour_name in ['부산 아난티코브 타운, 이터널저니']:
    driver = webdriver.Chrome('./chromedriver')
    driver.maximize_window()
    driver.implicitly_wait(4) 
    driver.get('https://map.kakao.com/')
    search(tour_name)
    names += [tour_name] * len(stars)
    driver.close()

In [ ]:
columns = ['name', 'user_id', 'score', 'review']
df_review = pd.DataFrame(columns=columns)
for name, user_id, star, review in zip(names, nicknames, stars, reviews):
    row = [name, user_id['data-userid'], star.text[0], review.find(name="span").text]
    series = pd.Series(row, index=df_review.columns)
    df_review = df_review.append(series, ignore_index=True)

print(df_review)